<a href="https://colab.research.google.com/github/eebjwa/SENSE_GEE_TEAM3/blob/main/SENTINEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#This will be our sentinel processing script
# Import earthengine API
import ee
# Authenticate and initialise 
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=rh-9VSjsVn86dOJb_0Ucm6FH-vQPHPim4_StxAbcWAM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AfDhmrgew3NikZjXr8nj8cBiwn3-1XMAqM2BxYqdP7JRgcU2sPud-f4OTW4

Successfully saved authorization token.


In [9]:
def ESAcloudMask(img):
    """
    European Space Agency (ESA) clouds from 'QA60', i.e. Quality Assessment band at 60m
     
    parsed by Nick Clinton
    """

    qa = img.select('QA60')

    # bits 10 and 11 are clouds and cirrus
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)

    # both flags set to zero indicates clear conditions.
    clear = qa.bitwiseAnd(cloudBitMask).eq(0).And(\
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    # clouds is not clear
    cloud = clear.Not().rename(['ESA_clouds'])

    # return the masked and scaled data.
    return img.addBands(cloud)

In [10]:
S2 = ee.ImageCollection("COPERNICUS/S2_SR")
sample_image = S2.filterBounds(ee.Geometry.Point(-71.0, 19.0)).sort('CLOUDY_PIXEL_PERCENTAGE').first()
#sample_image.getInfo()



In [11]:
import folium
!pip install geehydro # Life saver for plotting GEE stuff with Python!
import geehydro

cloudmasked_image = ESAcloudMask(sample_image)

map = folium.Map(location=[19.0,-71.0],zoom_start=9)
map.addLayer(cloudmasked_image,{'min':0,'max':2000,'bands':['B4','B3','B2']},'rbg')
map.addLayer(cloudmasked_image,{'min':0,'max':1,'bands':['ESA_clouds']},'cloud')
folium.LayerControl().add_to(map)
map

In [12]:
#train a classifier on the corine europe data set using Sentinel 2
training_image = S2.filterBounds(ee.Geometry.Point(11.4, 52.58)).sort('CLOUDY_PIXEL_PERCENTAGE').first()
roi = training_image.geometry()
#roi = ee.Geometry.Rectangle([10.4,52.0,12.5,53.5])
bands = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']

Corine_image = ee.ImageCollection("COPERNICUS/CORINE/V20/100m").first().clip(roi)


Corine_coll = ee.ImageCollection("COPERNICUS/CORINE/V20/100m")
Pal = Corine_coll.select('landcover').get('landcover_class_palette')
Pal_names = Corine_coll.get('landcover_class_names')



combined_image = ee.ImageCollection([training_image.select(bands),Corine_image])

randomPoints = ee.FeatureCollection.randomPoints(roi,1000);
#print(randomPoints.getInfo())
label = 'landcover'

Corine_samples = Corine_image.sampleRegions(randomPoints,geometries=True)
S2_samples = training_image.select(bands).sampleRegions(Corine_samples)
#print(Corine_samples.getInfo())
#print(S2_samples.getInfo())
#print(Corine_samples.getInfo())

#training = training_image.select(bands).sampleRegions(Corine_samples,properties =[label],scale=100)


trained = ee.Classifier.smileCart().train(S2_samples, label, bands)

classified = training_image.select(bands).classify(trained)

#classified.getInfo()




In [13]:
dataset = ee.Image('COPERNICUS/CORINE/V20/100m/2012');
landCover = dataset.select('landcover');

print(landCover.get('landcover_class_palette').getInfo())

['E6004D', 'FF0000', 'CC4DF2', 'CC0000', 'E6CCCC', 'E6CCE6', 'A600CC', 'A64DCC', 'FF4DFF', 'FFA6FF', 'FFE6FF', 'FFFFA8', 'FFFF00', 'E6E600', 'E68000', 'F2A64D', 'E6A600', 'E6E64D', 'FFE6A6', 'FFE64D', 'E6CC4D', 'F2CCA6', '80FF00', '00A600', '4DFF00', 'CCF24D', 'A6FF80', 'A6E64D', 'A6F200', 'E6E6E6', 'CCCCCC', 'CCFFCC', '000000', 'A6E6CC', 'A6A6FF', '4D4DFF', 'CCCCFF', 'E6E6FF', 'A6A6E6', '00CCF2', '80F2E6', '00FFA6', 'A6FFE6', 'E6F2FF']


In [49]:
landcover_vis = {'palette':['E6004D', 'FF0000', 'CC4DF2', 'CC0000', 'E6CCCC', 'E6CCE6', 'A600CC', 'A64DCC', 'FF4DFF', 'FFA6FF', 'FFE6FF', 'FFFFA8', 'FFFF00', 'E6E600', 'E68000', 'F2A64D', 'E6A600', 'E6E64D', 'FFE6A6', 'FFE64D', 'E6CC4D', 'F2CCA6', '80FF00', '00A600', '4DFF00', 'CCF24D', 'A6FF80', 'A6E64D', 'A6F200', 'E6E6E6', 'CCCCCC', 'CCFFCC', '000000', 'A6E6CC', 'A6A6FF', '4D4DFF', 'CCCCFF', 'E6E6FF', 'A6A6E6', '00CCF2', '80F2E6', '00FFA6', 'A6FFE6', 'E6F2FF']}
class_vals = landCover.get('landcover_class_values').getInfo()
new_classes = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43]

In [15]:
map = folium.Map(location=[52.58,11.4],zoom_start=9)

#roi = ee.Geometry.Rectangle([10.5,52,12.5,53.5])

map.addLayer(training_image, {'min':0,'max':2000,'bands': ['B4', 'B3', 'B2']}, 'image')
map.addLayer(Corine_image.select('landcover'), {}, 'landcover')
map.addLayer(randomPoints, {}, 'random points')
map.addLayer(classified,landcover_vis,'classification')
folium.LayerControl().add_to(map)
map

In [53]:

#Get composite image of S2 imagery over Hispianiola between jan-mar 2020 (filtered to cloud % <10)
HispComposite = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(ee.Geometry.Polygon(
        [[[-74.60668900677238, 20.088333497505946],
          [-74.60668900677238, 17.687361057873034],
          [-68.11376908489738, 17.687361057873034],
          [-68.11376908489738, 20.088333497505946]]])).filterDate('2020-01-10', '2020-03-20').filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)

#Get median image from the image collection (to reduce cloud impact)
HispCompositeMed = HispComposite.median()

#Run classification over whole island 
Hisp_classified = HispCompositeMed.select(bands).classify(trained)

#Define individual ROIs for Haiti and DR 
Haiti_roi = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Haiti'))
DR_roi = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Dominican Republic'))

#Clip classified image to each country (Haiti and DR)
Haiti_classified = Hisp_classified.clip(Haiti_roi)
DR_classified = Hisp_classified.clip(DR_roi)

#sample_image.getInfo()
map = folium.Map(location=[19.0,-71.0],zoom_start=9)
map.addLayer(sample_image,{'min':0,'max':2000,'bands':['B4','B3','B2']},'rbg')
map.addLayer(HispCompositeMed, {'min':0,'max':2000,'bands':['B4','B3','B2']},'Hispaniola_med')
map.addLayer(classified.randomVisualizer(), {}, 'classified_hisp')
map.addLayer(Haiti_classified.remap(class_vals, new_classes), {'min':0, 'max':43, 'palette':landcover_vis['palette']}, 'classified_Haiti')
map.addLayer(DR_classified.remap(class_vals, new_classes), {'min':0, 'max':43, 'palette':landcover_vis['palette']}, 'classified_DR')
folium.LayerControl().add_to(map)
map

In [ ]:
#Validate the classifier against the europe data set

In [ ]:
#run the classifier on a single image on hispaniola to sense-check

In [ ]:
#create a collection of classified images from sentinel from 2016 to now

In [ ]:
#make pretty plots